In [1]:
import os
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

# Path to the uploaded 'data' folder
data_dir = 'data/'

# List all subdirectories (classes)
classes = os.listdir(data_dir)
print("Classes:", classes)

# Create DataFrame for train and validation sets
data = []

# Iterate over classes and image files
for class_name in classes:
    class_dir = os.path.join(data_dir, class_name)
    for img_name in os.listdir(class_dir):
        img_path = os.path.join(class_dir, img_name)
        label = class_name  # Use class_name as label
        data.append((img_path, label))

# Convert lists to DataFrames
data_df = pd.DataFrame(data, columns=['image_path', 'label'])

# Split dataset into training and validation sets
train_df, val_df = train_test_split(data_df, test_size=0.2, stratify=data_df['label'], random_state=42)

# Data Augmentation and Loading
img_size = (100, 100)
batch_size = 32

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_dataframe(
    train_df,
    x_col='image_path',
    y_col='label',
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',  # Use categorical class mode
    shuffle=True
)

val_generator = val_datagen.flow_from_dataframe(
    val_df,
    x_col='image_path',
    y_col='label',
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',  # Use categorical class mode
    shuffle=False  # No need to shuffle validation data
)

# Model
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_size[0], img_size[1], 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(len(classes), activation='softmax')  # Output layer with softmax activation
])

# Compile Model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Training
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=10,
    validation_data=val_generator,
    validation_steps=len(val_generator)
)

# Save Model
model.save('mask_detection_model.h5')



Classes: ['without_mask', 'with_mask']
Found 6042 validated image filenames belonging to 2 classes.
Found 1511 validated image filenames belonging to 2 classes.



Epoch 1/10


149/189 [======================>.......] - ETA: 9s - loss: 0.4824 - accuracy: 0.7652

C:\ProgramData\anaconda3\Lib\site-packages\PIL\Image.py:970: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


189/189 [==============================] - 49s 252ms/step - loss: 0.4627 - accuracy: 0.7787 - val_loss: 0.2857 - val_accuracy: 0.8968
Epoch 2/10
189/189 [==============================] - 47s 250ms/step - loss: 0.3433 - accuracy: 0.8560 - val_loss: 0.1960 - val_accuracy: 0.9279
Epoch 3/10
189/189 [==============================] - 47s 249ms/step - loss: 0.2866 - accuracy: 0.8863 - val_loss: 0.2288 - val_accuracy: 0.9153
Epoch 4/10
189/189 [==============================] - 47s 249ms/step - loss: 0.2536 - accuracy: 0.9002 - val_loss: 0.1809 - val_accuracy: 0.9371
Epoch 5/10
189/189 [==============================] - 47s 248ms/step - loss: 0.2402 - accuracy: 0.9055 - val_loss: 0.1774 - val_accuracy: 0.9292
Epoch 6/10
189/189 [==============================] - 48s 255ms/step - loss: 0.2410 - accuracy: 0.9088 - val_loss: 0.1589 - val_accuracy: 0.9398
Epoch 7/10
189/189 [==============================] - 47s 251ms/step - loss: 0.2281 - accuracy: 0.9139 - val_loss: 0.1402 - val_accuracy: 0.9

C:\Users\Paarth Parag Doshi\AppData\Roaming\Python\Python311\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
